## decorater



- 闭包,通俗的讲即把函数名当作参数传递进去

In [30]:
## 先看一个闭包例子
def outer(x):

    def inner(y):
        return x + y

    return inner

print(outer(6)(5))
print("-"*100)

## 调用过程：
# 1) outer(6) 返回 inner(y)
# 2) inner(5) 这里发现x的值也能够传递到里层函数

11
----------------------------------------------------------------------------------------------------


- 先来一波最基本的装饰器

In [31]:
## wrapped 木有参数
def decorater(func):
    def wrapper():
        print(f"the func:{func.__name__} is already be decorated!")
        return func()
    return wrapper


@decorater
def test():
    print(f"this is test")

test()
print(test.__name__)

print("-"*100)

## wrapped 有固定参数

def decorater_2(func):
    def wrapper(value1,value2):
        print(f"the func with value:{func.__name__} is already be decorated!")
        return func(value1,value2)
    return wrapper


@decorater_2
def test_2(value1,value2):
    print(f"this is test_2 with value1:{value1}, value2:{value2}")

test_2(1,2)
print(test_2.__name__)

print('-'*100)
## 调用说明:调用test/test_2说明：
# 1) 先调用decorater 返回wrapper
# 2) 调用wrapper(value1,value2)

###############  这里有个细节！被装饰器装饰后的 wrapped 函数名字明显被改成 wrapper 的名字

### both wrapped wrapper 带参数
def decorater_3(value):
    def wrapper(func):
        def inner_wrapper(*args, **kwargs):
            print(args,kwargs)
            print(f">>> this is inner decorater!third called! value:{value}")
            return func(*args, **kwargs)

        print(f">>> this is second called! value:{value},{func}")
        return inner_wrapper

    print(f">>> this is first called! value:{value} ")
    return wrapper

@decorater_3(1)
def test_3(value_):
    print(f"this is test_3 with value:{value_}")

print("-"*100)
test_3(111)


## 调用过程解析
# 1)调用decorater_3(3) --> 返回 wrapper 此时value = 3
# 2)调用wrapper(test_3) -->返回 inner_wrapper(test_3)
# 3)执行 inner_wrapper

# 执行第一步其实时返回 wrapper(func(xx)) (第二次调用日志打印出来了)
# 第二部返回inner_wrapper，这里并不是inner_wrapper(*args, **kwargs) （通过改变inner_wrapper
# 的参数就可以知道了）
# 通过打印的日志可以知道,其实此时test_3(value_)已经变成调用 inner_wrapper(value_)(func = test_3)


# 注意必须返回 函数调用，即函数名,倘若改为 xxx(),则是直接返回结果.
# 会出现NoneType IS NOT CALLABLE错误

the func:test is already be decorated!
this is test
wrapper
----------------------------------------------------------------------------------------------------
the func with value:test_2 is already be decorated!
this is test_2 with value1:1, value2:2
wrapper
----------------------------------------------------------------------------------------------------
>>> this is first called! value:1 
>>> this is second called! value:1,<function test_3 at 0x050E5C90>
----------------------------------------------------------------------------------------------------
(111,) {}
>>> this is inner decorater!third called! value:1
this is test_3 with value:111


- 难点：@wraps

- 先了解 partial
- 这是functools,从上面的例子来看，加了装饰器后，函数的"__name__"等属性都被改变，变成装饰器的"__name__"
,作为一个坚持原则的我来说，是不能容忍自己随便被外界环境改变的！那么 @wraps他来了

In [32]:
import functools

## 普通装饰器
def decorater_1(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        """ this is decorator"""
        print(f">> i am decorator!")
        return func(*args, **kwargs)

    return wrapper

def decorater_1_1(func):
    def wrapper(*args, **kwargs):
        """ this is decorator"""
        print(f">> i am decorator!")
        return func(*args, **kwargs)

    return wrapper

@decorater_1
def test_1(value):
    """this is func"""
    print(f"i am func value:{value}")

@decorater_1_1
def test_1_1(value):
    """this is func"""
    print(f"i am func value:{value}")

test_1(11111)
print(test_1.__name__)
print(test_1.__doc__)

print('-'*100)

test_1_1(1111)
print(test_1_1.__name__)
print(test_1_1.__doc__)
# amazing! 加了@functools.wraps(func)后发现即使加了装饰器，也能保持原来的一些函数属性了！

>> i am decorator!
i am func value:11111
test_1
this is func
----------------------------------------------------------------------------------------------------
>> i am decorator!
i am func value:1111
wrapper
 this is decorator


In [33]:
# 再来一波带参数的装饰器
def decorator_2(value):
    def wrapper(func):
        def inner(*args,**kwargs):
            """this is decorator！！"""
            print(args,kwargs)
            print(f">> this is the inner !")
            return func(*args,**kwargs)

        print(f">> this is the second called ")
        return inner

    print(f">> this is the first called! value:{value}")
    return wrapper

@decorator_2(1)
def test_2(value):
    """this is func"""
    print(f">> this is func value:{value}")

print('-'*100)
test_2(1111)
print(test_2.__name__)
print(test_2.__doc__)  ## 同样是被改变了 function的一些属性~

print('-'*100)

def decorator_2_2(value):
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args,**kwargs):
            """this is decorator!"""
            print(args,kwargs)
            print(f">> this is the inner !")
            return func(*args,**kwargs)

        print(f">> this is the second called ")
        return inner

    print(f">> this is the first called! value:{value}")
    return wrapper

@decorator_2_2(1)
def test_2_2(value):
    """this is func"""
    print(f">> this is func value:{value}")

print('-'*100)
test_2_2(1111)
print(test_2_2.__name__)
print(test_2_2.__doc__)  ## 同样是被改变了 function的一些属性~



>> this is the first called! value:1
>> this is the second called 
----------------------------------------------------------------------------------------------------
(1111,) {}
>> this is the inner !
>> this is func value:1111
inner
this is decorator！！
----------------------------------------------------------------------------------------------------
>> this is the first called! value:1
>> this is the second called 
----------------------------------------------------------------------------------------------------
(1111,) {}
>> this is the inner !
>> this is func value:1111
test_2_2
this is func


- @wraps的实现原理其实就是利用了functool 的 partial，对其中的一些属性进行的重新赋值，具体请看
本项目的相关单元 
